In [13]:
import pandas as pd
from datasets import load_dataset
ds = load_dataset("gneubig/aime-1983-2024")["train"]

In [14]:
df = pd.DataFrame(ds)[:-14]

In [15]:
df["Question"]

0      Let $x$ , $y$ and $z$ all exceed $1$ and let $...
1      Let $f(x)=|x-p|+|x-15|+|x-p-15|$ , where $0 < ...
2      What is the product of the real roots of the e...
3      A machine-shop cutting tool has the shape of a...
4      Suppose that the sum of the squares of two com...
                             ...                        
914    Find the number of collections of $16$ distinc...
915    In $\triangle ABC$ with side lengths $AB = 13,...
916    Let $A$ be an acute angle such that $\tan A = ...
917    A cube-shaped container has vertices $A,$ $B,$...
918    For each positive integer $n$ let $a_n$ be the...
Name: Question, Length: 919, dtype: object

In [16]:
import litellm
import os
from dotenv import load_dotenv
load_dotenv(override=True)
from litellm import batch_completion
from litellm import completion  
import re 

os.environ["OPENAI_API_KEY"] = "NhBnln5YRzkZ69ufUvuvo2ZZ6IEQCpbQ"
# os.environ["TOGETHERAI_API_KEY"] = "tgp_v1_5ZFBneM2MDRlQrcI03AUbu7GtKbuUOSV1C2w0Kayc3k"

In [18]:
# response=completion(
#     model="together_ai/deepseek-ai/DeepSeek-R1", 
# messages=[{"role": "user", "content": "3 + 2*8 = ? Think and asnswer"}],
#     temperature = 0.6,
#     max_tokens = 2000,
#     # response_format={"type":"json_object"}
# )

In [20]:
# response_message = responses[0].choices[0].message

# # Check if the attribute exists before accessing it
# if hasattr(responses[0].choices[0].message, 'reasoning_content'):
#     print(response_message.reasoning_content)
# else:
#     print("The attribute 'reasoning_content' does not exist.")

In [21]:
def inf(start, end):
    problems_list = list(df["Question"][start:end])
    
    batch_prompt=[]
    for problem in problems_list:
        batch_prompt.append([{"role": "system", "content": "Please think step-by-step and give final answer using \\boxed{}, after taking modulo 1000. Answer by thinking as less as possible, but arriving at the solution."},{"role": "user", "content": problem }])
        
    responses=batch_completion(
        model="deepinfra/deepseek-ai/DeepSeek-R1", 
        messages=batch_prompt,
        temperature = 0.6,
        max_tokens = 8192,
        top_p = 0.95
    )

    return [response.choices[0].message.reasoning_content if hasattr(response.choices[0].message, 'reasoning_content') else "ETL" for response in responses]

In [22]:
def extract_boxed_texts(text):
    pattern = r'\\boxed{((?:[^{}]|{[^{}]*})+)}'
    matches = re.findall(pattern, text)
    if not matches:
        return -1000
    ans = []
    for content in matches:
        if content.isdigit():
            num = int(content)
            ans.append(num)

    if len(ans) > 0: return ans[-1]
    return matches[-1]    

In [ ]:
%%time

part_1 = inf(0, 200)
part_2 = inf(200, 400)
part_3 = inf(400, 600)
part_4 = inf(600, 800)
part_5 = inf(800, len(df))

In [ ]:
round_1 = part_1 + part_2 + part_3 + part_4 + part_5

In [ ]:
answers = [extract_boxed_texts(text) for text in round_1]

In [ ]:
idxs = []
for idx, (i, j) in enumerate(zip(answers, list(df["Answer"]))):
    if i==j:
        idxs.append(idx)

In [ ]:
df["correct_responses"] = [round_1[i] if i in idxs else "None" for i in range(len(df))]

In [ ]:
df[df["correct_responses"] == "None"].to_csv("prev_year_aime_round_1_failed.csv", index = None)
df[df["correct_responses"] != "None"].to_csv("SFT_samples_prev_year_aime_round_1.csv", index = None)

## Extra code - no need to run

## Xtra code

In [ ]:
idxs = [333, 348, 368, 406, 408, 495, 523, 529, 603, 696, 701, 745, 747, 769, 782, 815, 829]
len(idxs)

In [ ]:
final_filtered_problems = [final_json[i] for i in range(len(final_json)) if i not in idxs]
len(final_filtered_problems)

In [ ]:
"[/asy]" in str(final_filtered_problems)

In [ ]:
with open('AIME_full.json','w') as f:
    json.dump(final_complete,f)

In [ ]:
# with open('AIME_full.json','r') as f:
#     final_complete = json.load(f)

# final_questions = []

# for ele in final_complete:
#     final_questions.extend(extract_json_from_text(ele))


In [ ]:
len(final_questions)

In [ ]:
import pandas as pd
df=pd.DataFrame(final_json)

In [ ]:
df

In [ ]:
df = df[df["answer"] != -1000].reset_index(drop = True)

In [ ]:
df.to_csv('evan_chan_aime_mock_qns.csv',index=False)